In [208]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point, LineString, Polygon, MultiLineString, MultiPoint, MultiPolygon

from iduedu import get_intermodal_graph

C:\Program Files\JetBrains\PyCharm 2024.2.2\plugins\python-ce\helpers\pydev\_pydev_bundle\pydev_import_hook.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  module = self._system_import(name, *args, **kwargs)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [78]:
context_ter = gpd.read_file('context_ter.geojson')

In [196]:
natural = gpd.read_file('shl_natural_estimated.geojson')

In [185]:
def value_estimator(natural_tag: str) -> int:
    """Function estimates attraction based on tag"""
    
    natural_tag_router = {
        'beach': 5,
        'bay': 10,
        'spring': 2,
        'cape': 10,
        'clif': 7
    }
    
    result = natural_tag_router.get(natural_tag, 1)
    return result

In [198]:
tourism = ox.features_from_polygon(context_ter.union_all(), {'tourism': True})

In [200]:
tourism['tourism'].unique()

array(['hotel', 'viewpoint', 'attraction', 'museum', 'information',
       'artwork', 'picnic_site', 'camp_site', 'motel', 'guest_house'],
      dtype=object)

In [201]:
accommodation = tourism[tourism['tourism'].isin(['hotel', 'camp_site', 'motel', 'guest_house'])]

In [206]:
accommodation = accommodation[['tourism', 'geometry']]

In [207]:
accommodation.reset_index()

tourism  \
element id                         
node    380179312          hotel   
        1056680936         hotel   
        1179449583         hotel   
        1888235585         hotel   
        7353964075         hotel   
        7812529695     camp_site   
        7812529707     camp_site   
        11268054600        motel   
        11581674773        hotel   
way     38183949           hotel   
        116073249          hotel   
        650038216    guest_house   

                                                              geometry  
element id                                                              
node    380179312                            POINT (31.04647 59.94045)  
        1056680936                           POINT (31.03513 59.94645)  
        1179449583                           POINT (31.21456 59.87327)  
        1888235585                            POINT (30.98878 59.8887)  
        7353964075                           POINT (30.98659 59.87632)  
        7812529695                            POINT (31.1293 59.84639)  
        7812529707                           POINT (31.20158 59.80727)  
        11268054600                          POINT (31.06263 59.90699)  
        11581674773                          POINT (30.98981 59.87333)  
way     38183949     POLYGON ((31.0327 59.9467, 31.03251 59.94672, ...  
        116073249    POLYGON ((31.03446 59.9465, 31.03459 59.94648,...  
        650038216    POLYGON ((31.00121 59.92953, 31.00129 59.92961...

In [ ]:
G_public_t = get_all_public_transport_graph(polygon=bounds, clip_by_bounds=True,
                                            keep_geometry=False)